## Intro

With threads, 99% of the use cases an application programmer is likely to run into is the simple pattern of "spawning a bunch of independent threads and collecting the results in a queue". This chapter focuses on the `concurrent.futures.Executor` classes that encapsulate this pattern. 

We will also learn the concept of `futures` — objects representing the asynchronous execution of an operation (and foundational to the `asyncio` package), similar to JavaScript's `promises`.

## Concurrent Web Downloads

<span style="color:skyblue">***Concurrency is essential for efficient network I/O: instead of idly waiting for remote machines, the application should do something else until a response comes back***</span>.

We will learn 3 scripts to download 20 flags from the web: 
- `flags.py`: run downloads sequentially
- `flags_threadpool.py`: make concurrent downloads using `concurrent.futures`
- `flags_asyncio.py`: make concurrent downloads using `asyncio`

The sequential code will take the longest, while the 2 others make similar perfomance with much shorter time. And also, the order of the flags downloaded will be different in each code

### A sequential download script

In [ ]:
import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()  # country codes for the 20 most populous countries

BASE_URL = 'https://www.fluentpython.com/data/flags'  # The directory with the flag images
DEST_DIR = Path('downloaded')                         # Local directory where the images are saved.

def save_flag(img: bytes, filename: str) -> None:     # Save the img bytes to filename in the DEST_DIR
    (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:  # <6>
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = httpx.get(url, timeout=6.1,       # <7>
                     follow_redirects=True)  # <8>
    resp.raise_for_status()  # <9>
    return resp.content

def download_many(cc_list: list[str]) -> int:  # <10>
    for cc in sorted(cc_list):                 # <11>
        image = get_flag(cc)
        save_flag(image, f'{cc}.gif')
        print(cc, end=' ', flush=True)         # <12>
    return len(cc_list)

def main(downloader: Callable[[list[str]], int]) -> None:  # <13>
    DEST_DIR.mkdir(exist_ok=True)                          # <14>
    t0 = time.perf_counter()                               # <15>
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')

if __name__ == '__main__':
    main(download_many)     # <16>

## Launching Processes with `concurrent.futures`

## Experimenting with `Executor.map`

## Downloads with Progress Display and Error Handling

## Summary

## Further Reading